In [12]:
!pip install keybert

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
import pandas as pd
from keybert import KeyBERT
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [27]:

# preprocess a single tweet/text
def preprocess_text(text):
    tokens = word_tokenize(text)
    tokens = [word.lower() for word in tokens if word.isalpha()]
    stop_words = set(stopwords.words('english'))
    stop_words.add("embeddedurl")
    tokens = [word for word in tokens if word not in stop_words]

    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    return tokens


kw_model = KeyBERT()


df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/full_year.csv")
# df = df.head(100)
df['tokenized_text'] = df['clean_text'].apply(preprocess_text)

df['bigram_keyphrases'] = df['clean_text'].apply(lambda x: kw_model.extract_keywords(x, keyphrase_ngram_range=(1, 2)))
df = df[['clean_text', 'tokenized_text', 'religion', 'spirituality', 'bigram_keyphrases']]
df.to_csv('/content/drive/MyDrive/Colab Notebooks/data.csv', index=False)

In [50]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data.csv")
df.head(20)
print(df.bigram_keyphrases[33])
print(df.clean_text[33])


[('religion misoverstood', 0.5176), ('black families', 0.453), ('held megachurches', 0.4213), ('forgiveness racism', 0.4173), ('megachurches giving', 0.4054)]
__bigo__ religion misoverstood has held us back...too many megachurches not giving back and too much forgiveness of the racism. we need more black people making black families with focus on stem and ownership.


In [41]:
print(type(df['bigram_keyphrases'][33]))

<class 'str'>


In [51]:
import ast

# convert string representation of tuples into actual tuples
df['bigram_keyphrases'] = df['bigram_keyphrases'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else None)
df['cleaned_bigrams'] = df['bigram_keyphrases'].apply(lambda x: [item[0] for item in x] if x else [])

print(df.cleaned_bigrams[33])

df.to_csv('/content/drive/MyDrive/Colab Notebooks/with_clean_bigrams.csv', index=False)

['religion misoverstood', 'black families', 'held megachurches', 'forgiveness racism', 'megachurches giving']
